In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install tifffile
!pip install segmentation-models-pytorch

     |████████████████████████████████| 133kB 4.9MB/s 
     |████████████████████████████████| 18.1MB 196kB/s 
     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60962 sha256=35f27e67dcb9774a455954c0f732009556a6bc439265c4f55ef311785b322622
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12422 sha256=e684a488c31c5f03990d7903c7aa7476e51d2f7d403f9a31be0dcfdfd8b76e8b
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built pretrainedmodels efficientnet-pytorch


In [3]:
cd /content/drive/My Drive/Projet SatMap/Implementation

/content/drive/My Drive/Projet SatMap/Implementation


In [4]:
# Define the helper function
def decode_segmap(image, nc=21):
  
  label_colors = np.array([(0, 0, 0),  # 0=background
               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
               (128, 0, 0), (0, 128, 0), (128, 128, 0), (0, 0, 128), (128, 0, 128),
               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
               (0, 128, 128), (128, 128, 128), (64, 0, 0), (192, 0, 0), (64, 128, 0),
               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
               (192, 128, 0), (64, 0, 128), (192, 0, 128), (64, 128, 128), (192, 128, 128),
               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
               (0, 64, 0), (128, 64, 0), (0, 192, 0), (128, 192, 0), (0, 64, 128)])

  r = np.zeros_like(image).astype(np.uint8)
  g = np.zeros_like(image).astype(np.uint8)
  b = np.zeros_like(image).astype(np.uint8)
  
  for l in range(0, nc):
    idx = image == l
    r[idx] = label_colors[l, 0]
    g[idx] = label_colors[l, 1]
    b[idx] = label_colors[l, 2]
    
  rgb = np.stack([r, g, b], axis=2)
  return rgb

# Liste des images annotees
LIST_IMG_ID = ('6010_1_2', '6010_4_2', '6010_4_4', '6040_1_0', '6040_1_3', '6040_2_2', 
         '6040_4_4', '6060_2_3', '6070_2_3', '6090_2_0', '6100_1_3', '6100_2_2',
         '6100_2_3', '6110_1_2', '6110_3_1', '6110_4_0', '6120_2_0', '6120_2_2',
         '6140_1_2', '6140_3_1', '6150_2_3', '6160_2_1', '6170_0_4', '6170_2_4',
         '6170_4_1')
# ID des classes respectives : batiments, routes, arbres, rivieres, lacs 
LIST_POLY_TYPE = ('1', '3', '5', '7', '8')
IMG_ID = LIST_IMG_ID[17]
POLY_TYPE = LIST_POLY_TYPE[0]

from torchvision import transforms, datasets, models
import torch
from torch.autograd import Variable
import torch.nn as nn

import segmentation_models_pytorch as smp

import numpy as np

from PIL import Image
import shapely.affinity

import tifffile as tiff

import matplotlib.pyplot as plt

# Ouverture des fichiers
import bin.data_opening as op

data_opening = op.DataOpening()

#img = Image.open('./data/three_band/6120_2_2.tif')
#img_size = img.size
img, img_size = data_opening.get_tiff(IMG_ID)

x_max, y_min = data_opening.get_size(IMG_ID)
train_polygons = data_opening.get_polygons(IMG_ID, POLY_TYPE)

# Traitment des fichiers
import bin.treatment as tr

treatment = tr.Treatment()

x_scaler, y_scaler = treatment.get_scalers(img_size, x_max, y_min)

list_polygons = shapely.affinity.scale(train_polygons, xfact=x_scaler, yfact=y_scaler, origin=(0, 0, 0))

train_mask = treatment.get_mask_polygons(img_size, list_polygons)

img = treatment.get_img_rgb(img)
mask = train_mask

#fcn = models.segmentation.fcn_resnet101(pretrained=True).eval()
#fcn = models.segmentation.fcn_resnet101(num_classes=2).train()
fcn = smp.Unet("resnet18", encoder_weights="imagenet", classes=1, activation=None)

# Apply the transformations needed
import torchvision.transforms as T
X_transforms = T.Compose([T.Resize(256),
                          T.CenterCrop(224),
                          T.ToTensor(),
                          T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                          ])

X_t = T.Compose([T.Resize(256),
                 T.CenterCrop(224)
                 ])

Y_transforms = T.Compose([T.Resize(256),
                          T.CenterCrop(224),
                          T.ToTensor()
                          #T.Normalize((0.5,), (0.5,))
                          ])

X_train = X_transforms(Image.fromarray(img.astype('uint8'))).unsqueeze(0)
Y_train = Y_transforms(Image.fromarray(mask.astype('uint8'))).unsqueeze(0)
#X_train = X_transforms(Image.fromarray(img.astype('uint8')))
#Y_train = Y_transforms(Image.fromarray(mask.astype('uint8')))

test = X_t(Image.fromarray(img.astype('uint8')))
pixels = list(test.getdata())
width, height = test.size
pixels = [pixels[i * width:(i + 1) * width] for i in range(height)]

#print(pixels)
#print(Y_train)
#plt.imshow(X_train); plt.show()
#plt.imshow(Y_train); plt.show()

#criterion = nn.MSELoss(reduction='sum')
criterion = nn.MSELoss()

optimizer = torch.optim.SGD(fcn.parameters(), lr=1e-4, momentum=0.9)

for t in range(10):
    # Pass the input through the net
    Y_pred = fcn(X_train)['out']
    #print(out.shape)
    
    # Compute and print loss
    loss = criterion(Y_pred, Y_train)
    #if t % 100 == 99:
    print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

def tensor_to_image(tensor, mean=0, std=1):
    img = np.transpose(tensor.detach().numpy(), (1, 2, 0))
    img = (img*std+ mean)*255
    img = img.astype(np.uint8)
    return img 

#om = transforms.ToPILImage(X_train.squeeze())
#om = tensor_to_image(X_train.squeeze(0))

#om = tensor_to_image(Y_pred.squeeze(0))
#om = treatment.get_img_rgb(om.transpose(1,2,0))

om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
print(om)
print (om.shape)
#om = Image.fromarray(om)
#print (om.size)
#print(om)

rgb = decode_segmap(om)
plt.imshow(rgb); plt.show()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


TypeError: ignored